In [1]:
import pandas as pd
import tensorflow_hub as hub
import openai 
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
import ast
import cx_Oracle
import re

from pathlib import Path
import sys
sys.path.append('../')
import json
import requests


2024-09-03 14:06:53.789441: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 14:06:53.789511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 14:06:53.809988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 14:06:53.852734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 14:06:54.587065: W tensorflow/compiler/tf2

In [2]:
prefix = Path.cwd().parent.parent
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
prefix

PosixPath('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise')

In [121]:
table_csv = pd.read_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/tig/tables.csv')
sql_json = pd.read_json('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/tig/ambiguous_new_queries.json')

In [89]:
db_to_tables = {}
db_to_columns = {}
for i in range(len(table_csv)):
    db_table = table_csv['schema'][i].split(',')[0].split('-')
    db = db_table[0]
    table = db_table[1]
    if db not in db_to_tables.keys():
        db_to_tables[db] = {table:table_csv['schema'][i].split(',')[1:]}
        db_to_columns[db] = set(table_csv['schema'][i].split(',')[1:])
    else:
        db_to_tables[db][table] = table_csv['schema'][i].split(',')[1:]
        db_to_columns[db].update(table_csv['schema'][i].split(',')[1:])
set(db_to_tables.keys())

{'csail_stata_cinder',
 'csail_stata_glance',
 'csail_stata_heat',
 'csail_stata_neutron',
 'csail_stata_nova',
 'csail_stata_nova_api',
 'keystone'}

In [122]:
ar = []
for i in range(len(sql_json)):
    for db_id in db_to_tables.keys():
        if set(db_to_tables[db_id].keys())>=set(Parser(sql_json['sql'][i]).tables):
            print(f'{i}:{db_id}')
            df.loc[i, "db_id"] = db_id
            columns=[]
            for c in Parser(sql_json['sql'][i]).columns:
                if '.' in c:
                    columns.append(c.split('.')[1])
                else:
                    columns.append(c)
            print(set(columns)<= db_to_columns[db_id])
            if not set(columns)<= db_to_columns[db_id]:
                print(columns)
                print(sql_json['sql'][i])
#Parser(sql_json['sql'][0]).tables


0:csail_stata_nova
True
1:keystone
True
2:csail_stata_nova
True
3:csail_stata_nova
True
4:csail_stata_nova
True
5:csail_stata_nova
True
6:csail_stata_neutron
True
7:csail_stata_neutron
True
8:csail_stata_nova
True
9:csail_stata_nova
True
10:csail_stata_nova
True
11:csail_stata_neutron
True
12:csail_stata_neutron
True
13:csail_stata_nova
True
14:csail_stata_neutron
True
15:csail_stata_neutron
True
16:csail_stata_glance
True
17:csail_stata_neutron
True
18:csail_stata_neutron
True
19:csail_stata_nova
True
20:csail_stata_nova
True
21:csail_stata_nova
True
22:csail_stata_neutron
True
23:csail_stata_nova
True
24:csail_stata_nova
True
25:csail_stata_nova
True
26:csail_stata_nova
True
27:csail_stata_neutron
True
28:csail_stata_nova
True
29:csail_stata_nova
True
30:csail_stata_nova
True
31:csail_stata_neutron
True
32:csail_stata_nova
True
33:csail_stata_nova
True
34:csail_stata_nova
True
35:csail_stata_nova
True
36:csail_stata_neutron
True
37:csail_stata_neutron
True
38:csail_stata_nova
True
39

In [123]:

sql_json.to_json('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/tig/ambiguous_new_queries.json',orient='records')
#sql_json.to_json('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/tig/unique_new_queries.json',orient='records')

In [91]:
# Open the file in read mode
ar = []
with open(str(prefix) + '/data/tig/nimbus-1_2024-08-28.log', 'rb') as file:
    # Read the file in chunks
    chunk_size = 50000000#2500000  # 1 MB
    test = True
    while test:
        chunk = file.read(chunk_size)
        if not chunk:
            break
        # Process the chunk (e.g., print it)
        print(chunk[:1000])
        test = False
chunk = chunk.decode('utf-8', errors='ignore')

b"/usr/sbin/mysqld, Version: 5.5.63-MariaDB-1~trusty-wsrep (mariadb.org binary distribution, wsrep_25.25.r9949137). started with:\nTcp port: 3306  Unix socket: /var/run/mysqld/mysqld.sock\nTime                 Id Command    Argument\n240828 11:31:17\t87616 Query\tCOMMIT\n\t\t87616 Query\tROLLBACK\n\t\t87663 Query\tSELECT 1\n\t\t87693 Query\tROLLBACK\n\t\t87980 Query\tSELECT 1\n\t\t87980 Query\tSELECT instance_extra.numa_topology AS instance_extra_numa_topology, instance_extra.created_at AS instance_extra_created_at, instance_extra.updated_at AS instance_extra_updated_at, instance_extra.deleted_at AS instance_extra_deleted_at, instance_extra.deleted AS instance_extra_deleted, instance_extra.id AS instance_extra_id, instance_extra.instance_uuid AS instance_extra_instance_uuid \nFROM instance_extra \nWHERE instance_extra.deleted = 0 AND instance_extra.instance_uuid = 'cf62d565-491b-40a2-9747-3cc87204f6ee' \n LIMIT 1\n\t\t87980 Query\tROLLBACK\n\t\t87766 Query\tSELECT 1\n\t\t87663 Query\tS

In [99]:
a = re.split(r'\n[0-9\s:]+\s+Query\s+', chunk)

In [100]:
sum_of_tables = 0
sum_of_joins = 0
num_of_queries = 0
sql = []
for i in range(2, len(a)-1):
    if  not a[i].startswith('COMMIT') and not a[i].startswith('ROLLBACK') and not a[i].startswith('SET') and not a[i].startswith('SELECT 1') and 'SAVEPOINT' not in a[i]:
        try:
            number_of_tables = len(Parser(a[i]).tables)
            number_of_joins = a[i].count("JOIN")
            print(str(i)+ ":"+str(number_of_joins)+":"+str(number_of_tables))
            sum_of_tables += number_of_tables
            sum_of_joins += number_of_joins
            num_of_queries += 1
            sql.append(a[i])
        except Exception as e:
            print(e)

6:0:1
9:6:7
10:0:1
14:0:1
16:0:1
20:0:1
23:0:1
27:0:1
30:0:1
33:0:1
37:0:1
38:0:1
43:0:1
44:0:1
46:0:1
49:0:1
53:0:1
54:0:1
60:0:1
64:0:1
67:0:1
68:12:12
71:0:1
75:0:1
78:0:1
81:0:1
84:0:1
87:0:1
91:0:1
92:9:9
95:0:1
97:2:3
99:0:1
102:0:1
105:0:1
108:0:1
112:7:4
113:0:1
115:11:10
117:3:4
118:0:1
122:3:4
125:1:2
127:9:8
128:0:1
131:0:1
136:9:10
138:0:1
143:0:1
144:2:3
148:1:2
152:0:1
153:0:1
158:0:1
161:0:1
164:0:1
167:0:1
171:0:1
173:12:12
175:0:1
179:0:1
182:0:1
185:0:1
188:0:1
195:0:1
198:0:1
200:6:7
202:0:1
205:0:1
209:0:1
212:3:4
213:0:1
216:0:1
220:0:1
221:0:1
223:0:1
225:0:1
226:0:1
230:2:3
231:12:12
236:9:9
237:2:3
239:4:5
240:11:10
243:3:4
244:0:1
248:3:4
253:0:1
254:0:1
255:2:3
259:9:8
263:9:10
267:2:3
270:3:4
272:0:1
273:1:2
275:0:1
278:0:1
283:0:1
287:12:12
290:0:1
292:0:1
297:0:1
301:0:1
305:0:1
309:0:1
313:0:1
317:0:1
321:0:1
325:0:1
329:0:1
334:0:1
335:0:1
341:0:1
345:0:1
349:0:1
353:0:1
357:0:1
361:1:2
362:0:1
364:1:2
367:0:1
370:0:1
375:0:1
378:0:1
381:0:1
384:0:1
387:0

Not supported query type: SHOW VARIABLES LIKE 'sql_mode'
Not supported query type: show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
Not supported query type: SHOW VARIABLES LIKE 'sql_mode'


14235:0:1
14238:0:1
14241:0:1
14245:0:1
14246:0:1
14251:0:1
14253:0:1
14255:0:1
14256:0:1
14260:0:1
14263:1:2
14267:0:1
14270:0:1
14273:1:2
14276:0:1
Not supported query type!
14280:0:0
14281:0:0
Not supported query type!
14283:0:0
14284:0:0
14285:0:0
Not supported query type!
14293:2:3
14296:3:4
14298:0:1
14301:0:1
14302:0:1
14306:0:1
14308:0:1
14309:0:1
14311:0:1
14317:7:4
14318:0:1
14322:6:7
14324:3:4
14327:0:1
14328:0:1
14330:0:1
14332:12:12
14335:7:4
14339:9:9
14340:2:3
14342:11:10
14343:0:1
14344:3:4
14346:0:1
14350:3:4
14353:0:1
14354:0:1
14359:9:8
14360:3:4
14365:9:10
14368:3:4
14370:2:3
14373:1:2
14375:0:1
14376:0:1
14378:12:12
14382:0:1
14388:0:1
14391:0:1
14392:0:1
14396:1:2
14398:0:1
14399:0:1
14402:0:1
14403:0:1
14406:0:1
14408:0:1
14410:0:1
14412:0:1
14413:0:1
14420:0:1
14422:0:1
14423:0:1
14427:0:1
14431:0:1
14434:0:1
14437:0:1
14440:0:1
14445:0:1
14447:0:1
14448:1:2
14449:0:1
14452:0:1
14455:0:1
14458:0:1
14461:0:1
14463:0:1
14466:0:1
14469:0:1
14471:0:1
14472:0:1
14477

In [101]:
for a_ in sql:
    if 'Query' in a_:
        print(a_)
        print('____________________________________________________________________')

In [102]:
sum_of_tables/num_of_queries

1.8898821898821898

In [103]:
num_of_queries

14430

In [104]:
unique_sql = set(sql)

In [105]:
sum_of_tables = 0
sum_of_joins = 0
num_of_queries = 0
map ={}
only_more_than_one_table = []
for ele in unique_sql:
    try:
            number_of_tables = len(Parser(ele).tables)
            number_of_joins = ele.count("JOIN")
            print(str(num_of_queries)+ ":"+str(number_of_joins)+":"+str(number_of_tables))
            sum_of_tables += number_of_tables
            sum_of_joins += number_of_joins
            num_of_queries += 1
            if number_of_tables >1:
                only_more_than_one_table.append(ele)
            if str(number_of_tables) in map.keys():
                map[str(number_of_tables)] += 1
            else:
                map[str(number_of_tables)] = 1
    except Exception as e:
            print(e)

0:0:1
1:0:1
2:0:1
3:4:5
4:0:1
5:0:1
6:0:1
7:0:1
8:0:1
9:0:1
10:0:1
11:0:1
12:0:1
13:0:1
14:0:1
15:0:1
16:0:1
17:2:3
18:7:4
19:0:1
20:0:1
21:0:1
22:0:1
23:0:1
24:0:1
25:0:1
26:0:1
27:0:1
28:0:1
29:0:1
30:0:1
31:0:1
32:0:1
33:4:5
34:3:4
35:4:5
36:0:1
37:0:1
38:0:1
39:0:1
40:0:1
41:0:1
42:0:1
43:0:1
44:0:1
45:0:1
46:0:1
47:0:1
48:0:1
49:0:1
50:0:1
51:0:1
52:0:1
53:0:1
54:0:1
55:0:1
56:0:1
57:12:12
58:0:1
59:0:1
60:0:1
61:13:13
62:0:1
63:4:5
64:0:1
65:0:1
66:4:5
67:4:5
68:0:1
69:0:1
70:0:1
71:2:3
72:0:1
73:0:1
74:0:1
75:3:4
76:0:1
77:0:1
78:3:4
79:0:1
80:0:1
81:0:1
82:12:12
83:0:1
84:0:1
85:0:1
86:0:1
87:0:1
88:0:1
89:0:1
90:12:12
91:0:1
92:0:1
93:3:4
94:0:1
95:3:4
96:5:4
97:3:4
98:0:1
99:8:9
100:0:1
101:0:1
102:0:1
103:0:1
104:2:3
105:0:1
106:0:1
107:0:1
108:0:1
109:0:1
110:0:1
111:0:1
112:0:1
113:0:1
114:1:2
115:0:1
116:0:1
117:0:1
118:0:1
119:0:1
120:0:1
121:0:1
122:0:1
123:0:1
124:0:1
125:0:1
126:0:1
127:6:7
128:0:1
129:4:5
130:0:1
131:0:1
132:2:3
133:1:2
134:0:1
135:0:1
136:12:12
137:

In [106]:
sum_of_tables/num_of_queries

2.226629680998613

In [107]:
num_of_queries

3605

In [108]:
for i in range(1, 13):
    if str(i) in map.keys():
        print("number of tables in a query " + str(i) + " and they occured " + str(map[str(i)]) + " times")

number of tables in a query 1 and they occured 2618 times
number of tables in a query 2 and they occured 71 times
number of tables in a query 3 and they occured 79 times
number of tables in a query 4 and they occured 271 times
number of tables in a query 5 and they occured 309 times
number of tables in a query 6 and they occured 5 times
number of tables in a query 7 and they occured 106 times
number of tables in a query 8 and they occured 4 times
number of tables in a query 9 and they occured 6 times
number of tables in a query 10 and they occured 10 times
number of tables in a query 11 and they occured 1 times
number of tables in a query 12 and they occured 115 times


In [109]:
def count_queries(df, category):
    if category == 'selection': 
        m = [my_string.count('SELECT') for my_string in df['query']]
    elif category == 'filtering': 
        m = [len(re.findall(r'^(?=.*(WHERE|HAVING)(?=.*(=|>|<|IN|BETWEEN|AND|OR|NOT|LIKE)))', my_string)) for my_string in df['query']]
    elif category == 'aggregation': 
        m = [len(re.findall(r'SUM|AVG|MAX|MIN', my_string)) for my_string in df['query']]
    elif category == 'counting': 
        m = [my_string.count('COUNT') for my_string in df['query']]
    elif category == 'grouping':
        m = [my_string.count('GROUP BY') for my_string in df['query']]
    elif category == 'ordering': 
        m = [my_string.count('ORDER BY') for my_string in df['query']]
    elif category ==  'top-k': 
        m = [my_string.count('LIMIT') for my_string in df['query']]
        #m = [len(re.findall(r'^(?=.*ORDER BY )(?=.*LIMIT )', my_string)) for my_string in df['query']]
    elif category == 'join':
        m = [my_string.count('JOIN') for my_string in df['query']]
    elif category == 'intersect': 
        m = [my_string.count('INTERSECT') for my_string in df['query']]
    elif category == 'union': 
        m = [my_string.count('UNION') for my_string in df['query']]
    elif category == 'distinct': 
        m = [my_string.count('DISTINCT') for my_string in df['query']]
    elif category == 'pattern_matching': 
        m = [my_string.count('LIKE') for my_string in df['query']]
    elif category == 'nested':
        m = [len(re.findall(r'^(?=.*SELECT.*\(SELECT)', my_string)) for my_string in df['query']]
    else: 
        print('Invalid category!')
        
    return (sum(m), max(m))

In [110]:
unique_template_grams = []
unique_templates = []
for sql in only_more_than_one_table:
    exist = False
    grams = set(sql.split(' '))
    for template in unique_template_grams:
        if len(template.intersection(grams))/max(len(template), len(grams)) > 0.8:
            print(len(template.intersection(grams))/max(len(template), len(grams)))
            exist = True
    if not exist:
        unique_template_grams.append(grams)
        unique_templates.append(sql)

0.9965870307167235
0.9965870307167235
0.9664429530201343
0.9965870307167235
0.9965870307167235
0.9965870307167235
0.8358974358974359
0.9863945578231292
0.9863945578231292
0.9696969696969697
0.841025641025641
0.8899082568807339
0.8419689119170984
0.9965870307167235
0.9863945578231292
0.8358974358974359
0.9903846153846154
0.9844559585492227
0.9696969696969697
0.9965870307167235
0.9965870307167235
0.8899082568807339
0.8419689119170984
0.9965870307167235
0.9696969696969697
0.9863945578231292
0.8899082568807339
0.8419689119170984
0.9965870307167235
0.9696969696969697
0.8899082568807339
0.8419689119170984
0.9965870307167235
0.9428571428571428
0.9965870307167235
0.8899082568807339
0.8419689119170984
0.9965870307167235
0.9965870307167235
0.9965870307167235
0.9965870307167235
0.9903846153846154
0.8045977011494253
0.9696969696969697
0.9844559585492227
0.9965870307167235
0.9863945578231292
0.9855072463768116
0.9863945578231292
0.9903846153846154
0.9855072463768116
0.9696969696969697
0.99658703071

In [111]:
len(unique_templates)

34

In [112]:

query_category = ['selection', 'filtering', 'aggregation', 'counting','grouping', 
                  'ordering', 'top-k', 'join', 'intersect', 'union', 'distinct', 
                  'pattern_matching', 'nested']
df = pd.DataFrame(data={'sql': only_more_than_one_table, 'question': [""]*len(only_more_than_one_table), 'db_id':[""]*len(only_more_than_one_table)})
df2 = pd.DataFrame(data={'sql': unique_templates, 'question': [""]*len(unique_templates), 'db_id':[""]*len(unique_templates)})
df.to_json('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/tig/ambiguous_new_queries.json',orient='records')
df2.to_json('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/tig/unique_new_queries.json',orient='records')
#for c in query_category:
#    print(c + ':')
#    print(count_queries(df, c)) 
print('______________')

______________


In [27]:
len(df)

193

In [31]:
list(unique_sql)[58]

"SELECT cn.created_at, cn.updated_at, cn.deleted_at, cn.deleted, cn.id, cn.service_id, cn.host, cn.uuid, cn.vcpus, cn.memory_mb, cn.local_gb, cn.vcpus_used, cn.memory_mb_used, cn.local_gb_used, cn.hypervisor_type, cn.hypervisor_version, cn.hypervisor_hostname, cn.free_ram_mb, cn.free_disk_gb, cn.current_workload, cn.running_vms, cn.cpu_info, cn.disk_available_least, cn.host_ip, cn.supported_instances, cn.metrics, cn.pci_stats, cn.extra_resources, cn.stats, cn.numa_topology, cn.ram_allocation_ratio, cn.cpu_allocation_ratio, cn.disk_allocation_ratio, ram_inv.total AS inv_memory_mb, ram_inv.reserved AS inv_memory_mb_reserved, ram_inv.allocation_ratio AS inv_ram_allocation_ratio, ram_usage.used AS inv_memory_mb_used, cpu_inv.total AS inv_vcpus, cpu_inv.allocation_ratio AS inv_cpu_allocation_ratio, cpu_usage.used AS inv_vcpus_used, disk_inv.total AS inv_local_gb, disk_inv.reserved AS inv_local_gb_reserved, disk_inv.allocation_ratio AS inv_disk_allocation_ratio, disk_usage.used AS inv_local_

In [36]:
df.to_json(str(prefix) + '/data/OpenStack_queries.json', orient='records')

In [9]:
databases = {'csail_stata_cinder':[],'csail_stata_glance':[],'csail_stata_heat':[],'csail_stata_neutron':[] ,
             'csail_stata_nova':[] ,'csail_stata_nova_api':[],'keystone':[], 'moc_kaizen_cinder':[], 'moc_kaizen_glance':[] ,'moc_kaizen_neutron':[],
             'moc_kaizen_nova':[]  }

In [10]:
databases

{'csail_stata_cinder': [],
 'csail_stata_glance': [],
 'csail_stata_heat': [],
 'csail_stata_neutron': [],
 'csail_stata_nova': [],
 'csail_stata_nova_api': [],
 'keystone': [],
 'moc_kaizen_cinder': [],
 'moc_kaizen_glance': [],
 'moc_kaizen_neutron': [],
 'moc_kaizen_nova': []}

In [12]:
databases_creations = {'csail_stata_cinder':[],'csail_stata_glance':[],'csail_stata_heat':[],'csail_stata_neutron':[] ,
             'csail_stata_nova':[] ,'csail_stata_nova_api':[],'keystone':[], 'moc_kaizen_cinder':[], 'moc_kaizen_glance':[] ,'moc_kaizen_neutron':[],
             'moc_kaizen_nova':[]  }
for database in databases:# Open the file in read mode
    with open(str(prefix) + '/data/openstack/' + database+ '.sql', 'r') as file:
        # Read the file in chunks
        chunk_size = 2500000  # 1 MB
        test = True
        while test:
            chunk = file.read(chunk_size)
            test = False
        print(chunk.count("CREATE TABLE"))
        databases[database] = re.findall(r'CREATE TABLE `(.*?)`', chunk)
        table_creations = re.findall(r'CREATE TABLE (.*?) ENGINE=InnoDB', chunk, flags=re.DOTALL)
        databases_creations[database] = ['CREATE TABLE ' + tc for tc in table_creations]
        #print(table_names)
        print(table_creations[0])

25
`backups` (
  `created_at` datetime DEFAULT NULL,
  `updated_at` datetime DEFAULT NULL,
  `deleted_at` datetime DEFAULT NULL,
  `deleted` tinyint(1) DEFAULT NULL,
  `id` varchar(36) NOT NULL,
  `volume_id` varchar(36) NOT NULL,
  `user_id` varchar(255) DEFAULT NULL,
  `project_id` varchar(255) DEFAULT NULL,
  `host` varchar(255) DEFAULT NULL,
  `availability_zone` varchar(255) DEFAULT NULL,
  `display_name` varchar(255) DEFAULT NULL,
  `display_description` varchar(255) DEFAULT NULL,
  `container` varchar(255) DEFAULT NULL,
  `status` varchar(255) DEFAULT NULL,
  `fail_reason` varchar(255) DEFAULT NULL,
  `service_metadata` varchar(255) DEFAULT NULL,
  `service` varchar(255) DEFAULT NULL,
  `size` int(11) DEFAULT NULL,
  `object_count` int(11) DEFAULT NULL,
  `parent_id` varchar(36) DEFAULT NULL,
  `temp_volume_id` varchar(36) DEFAULT NULL,
  `temp_snapshot_id` varchar(36) DEFAULT NULL,
  `num_dependent_backups` int(11) DEFAULT NULL,
  `snapshot_id` varchar(36) DEFAULT NULL,
  `data

In [13]:
#df = pd.read_json(str(prefix) + '/data/OpenStack_queries.json')

In [14]:
databases_counter = {'csail_stata_cinder':0,'csail_stata_glance':0,'csail_stata_heat':0,'csail_stata_neutron':0 ,
             'csail_stata_nova':0 ,'csail_stata_nova_api':0,'keystone':0, 'moc_kaizen_cinder':0, 'moc_kaizen_glance':0 ,'moc_kaizen_neutron':0,
             'moc_kaizen_nova':0  }

In [54]:
for query in unique_sql:
    tables = Parser(query).tables
    for database in databases:
        if tables[0] in databases[database]:
            databases_counter[database] += 1

In [55]:
databases_counter

{'csail_stata_cinder': 213,
 'csail_stata_glance': 0,
 'csail_stata_heat': 64,
 'csail_stata_neutron': 178,
 'csail_stata_nova': 830,
 'csail_stata_nova_api': 0,
 'keystone': 84,
 'moc_kaizen_cinder': 213,
 'moc_kaizen_glance': 0,
 'moc_kaizen_neutron': 166,
 'moc_kaizen_nova': 831}

In [57]:
for query in df['query']:
    tables = Parser(query).tables
    for database in databases:
        if tables[0] in databases[database]:
            databases_counter[database] += 1

In [58]:
databases_counter

{'csail_stata_cinder': 2,
 'csail_stata_glance': 0,
 'csail_stata_heat': 1,
 'csail_stata_neutron': 88,
 'csail_stata_nova': 116,
 'csail_stata_nova_api': 0,
 'keystone': 7,
 'moc_kaizen_cinder': 2,
 'moc_kaizen_glance': 0,
 'moc_kaizen_neutron': 82,
 'moc_kaizen_nova': 117}

In [16]:
len(databases['moc_kaizen_nova'])

109

In [15]:
df_q = pd.DataFrame(data={'query':list(unique_sql)})

In [139]:
tables = []
questions = []
databases_qs = []
for query in unique_sql:
    tables.append(Parser(query).tables)
    questions.append('')
    database_q = []
    for database in reversed(databases.keys()):
        is_in = True
        for table in tables[len(tables)-1]:
            if table in databases[database]:
                is_in = is_in and True
            else:
                is_in = False
        if is_in:
            databases_counter[database] += 1
            database_q.append(database)
            break
    databases_qs.append(list(set(database_q)))
df_q['tables'] = tables
df_q['db_id'] = databases_qs
df_q['question'] = questions

In [74]:
df_q

query  \
0     SELECT anon_1.instances_created_at AS anon_1_instances_created_at, anon_1.instances_updated_at AS anon_1_instances_updated_at, anon_1.instances_deleted_at AS anon_1_instances_deleted_at, anon_1.instances_deleted AS anon_1_instances_deleted, anon_1.instances_id AS anon_1_instances_id, anon_1.instances_user_id AS anon_1_instances_user_id, anon_1.instances_project_id AS anon_1_instances_project_id, anon_1.instances_image_ref AS anon_1_instances_image_ref, anon_1.instances_kernel_id AS anon_1_instances_kernel_id, anon_1.instances_ramdisk_id AS anon_1_instances_ramdisk_id, anon_1.instances_hostname AS anon_1_instances_hostname, anon_1.instances_launch_index AS anon_1_instances_launch_index, anon_1.instances_key_name AS anon_1_instances_key_name, anon_1.instances_key_data AS anon_1_instances_key_data, anon_1.instances_power_state AS anon_1_instances_power_state, anon_1.instances_vm_state AS anon_1_instances_vm_state, anon_1.instances_task_state AS anon_1_instances_task_state, anon_1.instances_memory_mb AS anon_1_instances_memory_mb, anon_1.instances_vcpus AS anon_1_instances_vcpus, anon_1.instances_root_gb AS anon_1_instances_root_gb, anon_1.instances_ephemeral_gb AS anon_1_instances_ephemeral_gb, anon_1.instances_ephemeral_key_uuid AS anon_1_instances_ephemeral_key_uuid, anon_1.instances_host AS anon_1_instances_host, anon_1.instances_node AS anon_1_instances_node, anon_1.instances_instance_type_id AS anon_1_instances_instance_type_id, anon_1.instances_user_data AS anon_1_instances_user_data, anon_1.instances_reservation_id AS anon_1_instances_reservation_id, anon_1.instances_launched_at AS anon_1_instances_launched_at, anon_1.instances_terminated_at AS anon_1_instances_terminated_at, anon_1.instances_availability_zone AS anon_1_instances_availability_zone, anon_1.instances_display_name AS anon_1_instances_display_name, anon_1.instances_display_description AS anon_1_instances_display_description, anon_1.instances_launched_on AS anon_1_instances_launched_on, anon_1.instances_locked AS anon_1_instances_locked, anon_1.instances_locked_by AS anon_1_instances_locked_by, anon_1.instances_os_type AS anon_1_instances_os_type, anon_1.instances_architecture AS anon_1_instances_architecture, anon_1.instances_vm_mode AS anon_1_instances_vm_mode, anon_1.instances_uuid AS anon_1_instances_uuid, anon_1.instances_root_device_name AS anon_1_instances_root_device_name, anon_1.instances_default_ephemeral_device AS anon_1_instances_default_ephemeral_device, anon_1.instances_default_swap_device AS anon_1_instances_default_swap_device, anon_1.instances_config_drive AS anon_1_instances_config_drive, anon_1.instances_access_ip_v4 AS anon_1_instances_access_ip_v4, anon_1.instances_access_ip_v6 AS anon_1_instances_access_ip_v6, anon_1.instances_auto_disk_config AS anon_1_instances_auto_disk_config, anon_1.instances_progress AS anon_1_instances_progress, anon_1.instances_shutdown_terminate AS anon_1_instances_shutdown_terminate, anon_1.instances_disable_terminate AS anon_1_instances_disable_terminate, anon_1.instances_cell_name AS anon_1_instances_cell_name, anon_1.instances_internal_id AS anon_1_instances_internal_id, anon_1.instances_cleaned AS anon_1_instances_cleaned, instance_info_caches_1.created_at AS instance_info_caches_1_created_at, instance_info_caches_1.updated_at AS instance_info_caches_1_updated_at, instance_info_caches_1.deleted_at AS instance_info_caches_1_deleted_at, instance_info_caches_1.deleted AS instance_info_caches_1_deleted, instance_info_caches_1.id AS instance_info_caches_1_id, instance_info_caches_1.network_info AS instance_info_caches_1_network_info, instance_info_caches_1.instance_uuid AS instance_info_caches_1_instance_uuid, security_groups_1.created_at AS security_groups_1_created_at, security_groups_1.updated_at AS security_groups_1_updated_at, security_groups_1.deleted_at AS security_groups_1_deleted_at, security_groups_1.deleted AS security_groups_1_deleted, security_groups_1.id AS security_groups_1_id, securi

In [46]:
cnt_table_eq = 0
cnt_schema_eq = 0 
for database1 in databases:
    for database2 in databases:
        if database1 == database2:
            break
        for table in databases[database1]:
            if table in databases[database2]:
                cnt_table_eq +=1
                cnt_schema_eq += int(databases_creations[database1] == databases_creations[database2])

In [47]:
#how many schemas are the same
cnt_schema_eq

0

In [48]:
#how many tables have the same name in different databases
cnt_table_eq

356

In [22]:
databases_schemas = {'csail_stata_cinder':{},'csail_stata_glance':{},'csail_stata_heat':{},'csail_stata_neutron':{} ,
             'csail_stata_nova':{} ,'csail_stata_nova_api':{},'keystone':{}, 'moc_kaizen_cinder':{}, 'moc_kaizen_glance':{} ,'moc_kaizen_neutron':{},
             'moc_kaizen_nova':{} }
for database in databases:
    for t in databases[database]:
        databases_schemas[database][t] = []
        for database in databases_schemas:
for i in range(len(databases_creations[database])):
    databases_schemas[database][databases[database][i]] = re.findall(r'`([^`]+)`', databases_creations[database][i])

In [17]:
columns = []
for query in df_q['query']:
    column = {}
    old_t = ''
    for tac in Parser(query).columns:
        print(tac)
        table_and_column = tac.split('.')
        if len(table_and_column)<2:
            break
        t = table_and_column[0]
        c = table_and_column[1]
        if old_t == '' or old_t != t:
            column[t] = []
            old_t = t
        column[t].append(c)
    columns.append(column)
columns[0]
            

instance_extra.numa_topology
instance_extra.created_at
instance_extra.updated_at
instance_extra.deleted_at
instance_extra.deleted
instance_extra.id
instance_extra.instance_uuid
instance_extra.numa_topology
instance_extra.created_at
instance_extra.updated_at
instance_extra.deleted_at
instance_extra.deleted
instance_extra.id
instance_extra.instance_uuid
heartbeat_timestamp
services.created_at
services.updated_at
services.deleted_at
services.deleted
services.id
services.host
services.binary
services.topic
services.report_count
services.disabled
services.disabled_reason
services.last_seen_up
services.forced_down
services.version
agents.id
agents.agent_type
agents.binary
agents.topic
agents.host
agents.availability_zone
agents.admin_state_up
agents.created_at
agents.started_at
agents.heartbeat_timestamp
agents.description
agents.configurations
agents.resource_versions
agents.load
instance_extra.numa_topology
instance_extra.created_at
instance_extra.updated_at
instance_extra.deleted_at
insta

{'instance_extra': ['numa_topology',
  'created_at',
  'updated_at',
  'deleted_at',
  'deleted',
  'id',
  'instance_uuid']}

In [50]:
db_id = []
db_mis_index = []
for i in range(len(columns)):
    #print(column)
    for database in databases_schemas:
        is_in = True
        #print(database)
        for t in columns[i]:
            #print(t)
            if t not in databases_schemas[database].keys():
                is_in = False
                break
            if len(set(databases_schemas[database][t]).intersection(set(columns[i][t])))== len(set(columns[i][t])):
                is_in = is_in and True
            else:
                is_in = False
                break
            #for i in range(len(databases[database])):
            #    if t == databases[database][i]:
            #        #print(databases[database][i])
            #        if all([c in databases_creations[database][i] for c in column[t]]):
            #            is_in = is_in and True
            #            break
            #        else:
            #            is_in = False
            #            break
        #print(is_in)
        if is_in:
            db_id.append(database)
            break
    if not is_in:
        db_mis_index.append(i)
        print(i)

3
22
35
44
47
51
58
75
97
134
139
143
144
147
158
162
198
212
215
221
222
235
240
247
251
267
271
277
280
281
314
316
322
323
337
338
339
341
352
364
375
382
388
392
393
397
400
402
405
414
424
442
443
453
455
463
491
543
581
589
598
606
610
620
641
659
683
684
693
726
732
735
745
760
762
766
774
775
790
794
807
817
825
828
854
856
871
872
875
895
908
910
917
920
921
924
933
939
1029
1043
1053
1059
1060


In [43]:
len(db_id)

3478

In [51]:
len(db_mis_index)

103

In [52]:
len(db_id)

983

In [53]:
len(columns)

1086

In [44]:
3478/938

3.7078891257995736